In [89]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter, defaultdict, deque
import json
import itertools as it
import os
import utils
from utils import Labeler
import statsmodels
from scipy import stats

In [90]:
%run preprocess.ipynb

EXP3_N_PARTICIPANT = 112
EXP3_N_FORMAT = 5
EXP3_N_CAUGHT = 5
EXP3_N_REMAIN = 102
EXP3_N_RT_CLIP = 37
EXP3_THRESHOLD_RT_CLIP = 12.174455924796257
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


preprocess.ipynb:45: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  "outputs": [],


# Cue quiz

In [91]:
tbl = final_qdf.groupby('feedback').perfect.value_counts().unstack().T
tbl = tbl.rename(columns={False: 'nofb', True: 'fb'})


In [92]:
n_perfect = tbl.ix[True]
counts = tbl.sum(0)
for idx, prop in (n_perfect / counts).items():
    writevar('QUIZ_{}_P'.format(idx.upper()),
             '{:2.0f}'.format(prop * 100))

EXP3_QUIZ_NOFB_P = 21
EXP3_QUIZ_FB_P = 22


/usr/local/lib/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


## Stats

In [53]:
%%R -i counts -i n_perfect -o p1 -o p2 -o p3
p1 = prop.test(n_perfect[1], counts[1], p=1/6, alternative='greater')
p2 = prop.test(n_perfect[2], counts[2], p=1/6, alternative='greater')
p3 = prop.test(n_perfect, counts, alternative='less')
print(p1)
print(p2)
print(p3)


	1-sample proportions test with continuity correction

data:  n_perfect[1] out of counts[1], null probability 1/6
X-squared = 0.46538, df = 1, p-value = 0.2476
alternative hypothesis: true p is greater than 0.1666667
95 percent confidence interval:
 0.1261815 1.0000000
sample estimates:
        p 
0.2115385 


	1-sample proportions test with continuity correction

data:  n_perfect[2] out of counts[2], null probability 1/6
X-squared = 0.8, df = 1, p-value = 0.1855
alternative hypothesis: true p is greater than 0.1666667
95 percent confidence interval:
 0.1341913 1.0000000
sample estimates:
        p 
0.2244898 


	2-sample test for equality of proportions with continuity correction

data:  n_perfect out of counts
X-squared = 7.3575e-31, df = 1, p-value = 0.5
alternative hypothesis: less
95 percent confidence interval:
 -1.000000  0.135243
sample estimates:
   prop 1    prop 2 
0.2115385 0.2244898 



In [55]:
comps = {
    'nofb_null': {k: ri2py(v) for k, v in p1.items()},
    'fb_null': {k: ri2py(v) for k, v in p2.items()},
    'fb_nofb': {k: ri2py(v) for k, v in p3.items()},
}
extract = ['estimate', 'statistic', 'p.value', 'parameter']
tbl = pd.DataFrame(
    {e: [vals[e] for vals in comps.values()] for e in extract},
    index = comps.keys()
)
tex_chi2 = r"$\chi^2({parameter:.0f})={statistic:.3f},\ {signif}$"

tbl = tbl.applymap(lambda x: x[0] if len(x) == 1 else x[1] - x[0])
tbl = tbl.rename(columns=reformat_name)
tbl['signif'] = tbl[reformat_name('p_value')].apply(pval)
save_analysis(tbl, 'QUIZ_CHI_{idx}', tex_chi2)

EXP2_QUIZ_CHI_FB_NOFB = $\chi^2(1)=0.000,\ p = 0.50$


<IPython.core.display.Latex object>

EXP2_QUIZ_CHI_NOFB_NULL = $\chi^2(1)=0.465,\ p = 0.25$


<IPython.core.display.Latex object>

EXP2_QUIZ_CHI_FB_NULL = $\chi^2(1)=0.800,\ p = 0.19$


<IPython.core.display.Latex object>